In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import UnstructuredPDFLoader
# from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from coexpert.params import OPENAI_API_KEY


In [ ]:
path = './raw_data'
loader = PyPDFDirectoryLoader(path)
documents = loader.load()
print(documents)

# split the documents in small chunks
# Change the chunk_size and chunk_overlap as needed
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(documents)

pages = all_splits

# Replace book.pdf with any pdf of your choice
# loader = UnstructuredPDFLoader("../../raw_data/_51113879-001A_NG4_Tachy_CRT-D_PTM_en_S.pdf")
# pages = loader.load_and_split()


In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
docsearch = Chroma.from_documents(pages, embeddings).as_retriever()


In [ ]:
# Choose any query of your choice
query = "Give me the dimensions of defibrilators?"
# query = "Give me position of kamasutra"
docs = docsearch.get_relevant_documents(query)
# chain = load_qa_chain(OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY), chain_type="stuff")
chain = load_qa_chain(ChatOpenAI(
    temperature=0, openai_api_key=OPENAI_API_KEY), chain_type="stuff")
output = chain.run(input_documents=docs, question=query)
print(output)
